### Este notebook es para visualizar de forma rápida las estadísticas del Grado de Matemáticas de la UNED.

Para saltar directamente a las gráficas puedes usar los siguientes links:

[Tasa media de exito de examenes realizados](#TDEE) = $\frac{\text{Nº de examenes aprobados}}{\text{Nº de examenes realizados}}$

[Tasa media de exito de examenes realizados agrupada por cursos](#TDEEC)

[Porcentaje medio de alumnos que realizan 2 examenes](#P2E)

[Porcentaje medio de alumnos que se matriculan por 3ª vez](#M3)

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# line for plotting in SVG format
# %config InlineBackend.figure_format = 'svg'

import plotly
import plotly.offline as py
import plotly.graph_objs as go
# run at the start of every ipython notebook to use plotly.offline
py.init_notebook_mode(connected=True) # this injects the plotly.js source files into the notebook

In [2]:
def subject_name(subject_id):
    return original_data[original_data['ID'] == str(subject_id)].iloc[0, 4]

def success_rate(i, year):
    mask = (original_data['CURSO'] == year) & (original_data['ID'] == str(i))
    return float(original_data[mask]['Nº EXÁM. APTOS']/original_data[mask]['Nº EXÁM. REALIZADOS'])

def subjects_by_curso(c):
    return [element for element in identifiers if element[4] == str(c)]

def subject_years(i):
    return np.array(original_data.loc[original_data['ID'] == str(i), 'CURSO'])

def curso_subject(subject_id):
    return str(subject_id)[4]

He descargado previamente los archivos *xls* de la web de la UNED y los he abierto con un programa de hojas de calculo y vuelto a guardar como csv. Ahora los abrimos y los unimos en un solo Dataframe:

In [3]:
DIRECTORY = r'/Users/Dani/Documents/Desarrollo/Data_analysis/UNED/UNED-Statistics-Mathematics/input'
original_data = pd.DataFrame()
list_ = []
files = [f for f in os.listdir(DIRECTORY) if f[0] is not '.']
for file in files:
    df = pd.read_csv(os.path.join(DIRECTORY, file), header=2, decimal=',')
    list_.append(df)
original_data = pd.concat(list_)
data = original_data.copy()
identifiers = original_data.ID.unique()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 0 to 49
Data columns (total 55 columns):
CURSO                       240 non-null int64
TITULACIÓN                  240 non-null object
COD. TITULACIÓN             240 non-null int64
ID                          240 non-null object
ASIGNATURA                  240 non-null object
ESTUD. MATRICULADOS         240 non-null int64
CRÉDITOS ASIG.              240 non-null int64
CRÉDITOS MATRICULADOS       240 non-null int64
CRÉD. MATRI. 1ª VEZ         240 non-null int64
CRÉD. MATRI. 2ª VEZ         240 non-null int64
CRÉD. MATRI. 3ª VEZ         240 non-null int64
CRÉDITOS PRESENTADOS        240 non-null int64
CRÉDITOS SUPERADOS          240 non-null int64
CRÉDITOS RECONOCIDOS        240 non-null int64
TASA EVALUACIÓN             157 non-null float64
VAR. % TASA EVAL.           157 non-null float64
TASA ÉXITO                  157 non-null float64
VAR. % TASA ÉXIT.           157 non-null float64
TASA ÉXITO EXÁM. REALIZ.    157 non-nul

Podemos ver que algunas columnas no tienen todos los datos. Esto se debe a que hay años en los que las asignaturas no han sido ofertadas o no han tenido estudiantes. Eliminaremos estas filas del dataframe así como las que no nos interesen o sean redundantes.

In [5]:
# Deleting columns with no students data
mask = data['ESTUD. MATRICULADOS'] == 0

# Deleting non interesting columns
data = data[~mask]
columns_to_drop = [
    'COD. TITULACIÓN',
    '% ÉXITO 1PPF',
    '% ÉXITO 2PPJ',
    '% ÉXITO 1PPS',
    '% ÉXITO 2PPS',
    '% ÉXITO DIC.',
    'FECHA DESCARGA EXCEL',
    'Unnamed: 54',
    'Nº EXÁM. REALIZADOS 1PPF',
    'Nº EXÁM. REALIZADOS 2PPJ',
    'Nº EXÁM. REALIZADOS 1PPS',
    'Nº EXÁM. REALIZADOS 2PPS',
    'Nº EXÁM. APTOS 1PPF',
    'Nº EXÁM. APTOS 2PPJ',
    'Nº EXÁM. APTOS 1PPS',
    'Nº EXÁM. APTOS 2PPS',
    'Nº EXÁM. REALIZADOS DIC',
    'Nº EXÁM. APTOS DIC',
    'TASA RECONCIMIENTO',
    'CRÉDITOS RECONOCIDOS',
    'VAR. % TASA EVAL.',
    'VAR. % TASA ÉXIT.',
    'VAR. % ÉXITO EXÁM.',
    'VAR. % TASA REND.',
    'TASA EVALUACIÓN',
    'TASA RENDIMIENTO',
    '% ESTUD. 3 EXÁM.',
    '% ESTUD. 4 EXÁM.',
    '% ESTUD. 5 EXÁM.',
    'TASA ÉXITO EXÁM. REALIZ.'
]
data.drop(columns_to_drop, axis=1, inplace=True)

Creamos columnas para obtener una mayor precisión, puesto que los datos que vienen en el archivo csv estan truncados para 2 decimales y podemos calcular fácilmente algunos de ellos con los datos que nos dan. Después borramos algunos datos más que no necesitamos. 

In [6]:
data['Tasa exito examen'] = data['Nº EXÁM. APTOS']/data['Nº EXÁM. REALIZADOS']
data['% creditos matric. 1 vez'] = data['CRÉD. MATRI. 1ª VEZ']/data['CRÉDITOS MATRICULADOS']
data['% creditos matric. 2 vez'] = data['CRÉD. MATRI. 2ª VEZ']/data['CRÉDITOS MATRICULADOS']
data['% creditos matric. 3 vez'] = data['CRÉD. MATRI. 3ª VEZ']/data['CRÉDITOS MATRICULADOS']
data['Tasa exito'] = data['CRÉDITOS SUPERADOS']/data['CRÉDITOS PRESENTADOS']
data['Ratio tasas de exito'] = data['Tasa exito examen'] / data['Tasa exito']
columns_to_drop = [
    'CRÉD. MATRI. 1ª VEZ',
    'CRÉD. MATRI. 2ª VEZ',
    'CRÉD. MATRI. 3ª VEZ',
    'Nº EXÁM. APTOS',
    'Nº EXÁM. REALIZADOS'
]
data.drop(columns_to_drop, axis=1, inplace=True)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157 entries, 0 to 49
Data columns (total 26 columns):
CURSO                       157 non-null int64
TITULACIÓN                  157 non-null object
ID                          157 non-null object
ASIGNATURA                  157 non-null object
ESTUD. MATRICULADOS         157 non-null int64
CRÉDITOS ASIG.              157 non-null int64
CRÉDITOS MATRICULADOS       157 non-null int64
CRÉDITOS PRESENTADOS        157 non-null int64
CRÉDITOS SUPERADOS          157 non-null int64
TASA ÉXITO                  157 non-null float64
NOTA MEDIA                  157 non-null float64
% SUSPENSOS                 157 non-null float64
% APROBADOS                 157 non-null float64
% NOTABLES                  157 non-null float64
% SOBRESALIENTES            157 non-null float64
% MATRÍCULAS H.             157 non-null float64
% ESTUD. 0 EXÁM.            157 non-null float64
% ESTUD. 1 EXÁM.            157 non-null float64
% ESTUD. 2 EXÁM.            1

Creamos un indice jerarquico para después agrupar por asignaturas y años.

In [8]:
df = data.set_index(['ID', 'CURSO'])

Agrupamos por asignatura calculando las medias de los años anteriores. No todas las asignaturas tienen la misma cantidad de datos.

In [9]:
medias = df.mean(level='ID')
varianzas = df.var(level='ID')
medias['Asignatura'] = pd.Series([subject_name(i) for i in medias.index.values], index=medias.index)
medias['Curso'] = pd.Series([curso_subject(i) for i in medias.index.values], index=medias.index)

Dibujamos la grafica para la <a id='TDEE'>Tasa media de exito de examenes realizados</a> ordenando previamente el dataframe de menor a mayor.

In [10]:
medias.sort_values(by='Tasa exito examen', ascending=False, inplace=True)
trace = go.Bar(
    x = medias['Tasa exito examen'].values,
    y = medias['Asignatura'].values,
    text = np.arange(medias['Asignatura'].size, 0, -1),
    orientation='h'
)
d = [trace]
layout = go.Layout(
    title='Tasa media de éxito en exámenes realizados',
    margin = go.Margin(l=300),
    height = 1000,
    font=dict(size=12)
)
fig = go.Figure(data=d, layout=layout)
py.iplot(fig)

Personalmente, esta es la forma en la que siempre compruebo la dificultad de las asignaturas, puesto que muy frecuentemente no realizo las pruebas no presenciales y la nota que saco en el examen es mi nota definitiva.

Para ver rápidamente las asignaturas más dificiles de cada curso, podemos dibujar la gráfica <a id='TDEEC'>por grupos</a>.

In [11]:
medias.sort_values(by='Tasa exito examen', inplace=True)
traces = []
for curso in range(1,5):
    t = go.Bar(
        y = medias.loc[medias['Curso'] == str(curso), 'Tasa exito examen'].values,
        x = medias.loc[medias['Curso'] == str(curso), 'Asignatura'].values,
        text = np.arange(1, medias.loc[medias['Curso'] == str(curso), 'Asignatura'].size + 1),
        name = str(curso) + 'º'
    )
    traces.append(t)

d = [traces]
layout = go.Layout(
    title='Tasa media de éxito en exámenes realizados por curso',
    margin = go.Margin(b=200, t=20),
    height = 700,
    font=dict(size=8),
    barmode='group'
)
fig = go.Figure(data=traces, layout=layout)
py.iplot(fig)

La *Tasa de exito*(TDE) y la *Tasa de exito en exámenes*(TDEE) son diferentes, ya que no todos los alumnos que suspenden el examen suspenden la asignatura, pero todos los alumnos que suspenden la asignatura habrán suspendido también el examen. Por lo tanto, la TDE siempre será mayor que la TDEE.

Esto puede dar lugar a que el examen, como medida de dificultad, sea en unas asignaturas más importante que en otras. Las asignaturas que tengan un ratio TDEE:TDE bajo serán en las que más alumnos hayan aprobado sin haber aprobado el examen, por lo tanto, serán asignaturas en las que los trabajos "extra", como por ejemplo las Pruebas no presenciales, tendrán mayor importancia.

A continuación ordenamos el dataframe segun este ratio y dibujamos la gráfica para saber en cuales asignaturas la nota del examen ha sido crucial.

In [12]:
medias.sort_values(by='Ratio tasas de exito', inplace=True)
trace = go.Bar(
    y = medias['Ratio tasas de exito'].values,
    x = medias['Asignatura'].values,
    text = np.arange(1,medias['Asignatura'].size + 1)
)
d = [trace]
layout = go.Layout(
    title="Ratio 'Tasa de exito examen' / 'Tasa de exito'",
    margin = go.Margin(b=300, t=20),
    height = 700,
    font=dict(size=8)
)
fig = go.Figure(data=d, layout=layout)
py.iplot(fig)

Porcentaje de alumnos que <a id='P2E'\>realizan 2 exámenes<a>

In [13]:
medias.sort_values(by='% ESTUD. 2 EXÁM.', inplace=True)
trace = go.Bar(
    y = medias['% ESTUD. 2 EXÁM.'].values,
    x = medias['Asignatura'].values,
    text = np.arange(1,medias['Asignatura'].size + 1)
)
d = [trace]
layout = go.Layout(
    title="Porcentaje de alumnos que realizan 2 exámenes",
    margin = go.Margin(b=300, t=20),
    height = 700,
    font=dict(size=8)
)
fig = go.Figure(data=d, layout=layout)
py.iplot(fig)

Porcentaje medio de alumnos que <a id='M3'\>se matriculan por 3ª vez<a>.

In [14]:
medias.sort_values(by='% creditos matric. 3 vez', inplace=True)
trace = go.Bar(
    y = medias['% creditos matric. 3 vez'].values,
    x = medias['Asignatura'].values,
    text = np.arange(1,medias['Asignatura'].size + 1)
)
d = [trace]
layout = go.Layout(
    title="Porcentaje de alumnos que se matriculan por 3ª vez",
    margin = go.Margin(b=300, t=20),
    height = 700,
    font=dict(size=8)
)
fig = go.Figure(data=d, layout=layout)
py.iplot(fig)